<a href="https://colab.research.google.com/github/Yuming293/personal1/blob/main/SDW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# @title 开始安装基础文件

%cd /content/
!sudo python /content/nologs.py
clear_output()
inf('\u2714 Done','success', '50px')

In [0]:
#@title  _启动SD_
%cd /content/
!sudo python /content/SW_run.py

#wise = "--api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers --disable-console-progressbars --theme dark --upcast-sampling"   
#!python /content/webui.py $wise